In [2]:
from dotenv import load_dotenv
import numpy as np
import os
import pymysql
import pandas as pd

In [3]:
load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT"))
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_DATABASE = os.getenv("DB_NAME")

In [36]:
try:
    # Establish a connection to the database
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_DATABASE
    )

        # Create a cursor object to interact with the database
    cursor = connection.cursor()
except pymysql.Error as e:
    print(f"Error connecting to the database: {e}")


In [38]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS predictions(
    location INT,
    datetime DATETIME,
    passenger_count INT,
    percentile DOUBLE,
    PRIMARY KEY (location, datetime)
    );
""")

0

In [15]:
df = pd.read_csv('predictions.csv')

In [39]:
df = df.set_index('item_id')

In [40]:
df['percentile'] = df.groupby(level='item_id')['mean'].rank(pct=True)

In [41]:
df = df.reset_index()

In [42]:
insert_query = """
INSERT IGNORE INTO predictions (
    location,
    datetime,
    passenger_count,
    percentile
)
VALUES (%s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    passenger_count = VALUES(passenger_count),
    percentile = VALUES(percentile);
"""

data = [] # Data to be used

for i, row in df.iterrows():
    data += [(
            row['item_id'],
            row['timestamp'],
            row['mean'],
            row['percentile'],
    )]
try:
    cursor.executemany(insert_query, data)
except pymysql.MySQLError as e:
    print(f"Error: {e}")
    connection.rollback()

connection.commit()

In [45]:

try:
    cursor.execute("""SELECT *
                FROM predictions;
    """)
    uploads = [row for row in cursor.fetchall()]
except pymysql.MySQLError as e:
    print(f"Error: {e}")

print(len(uploads))

198720


In [57]:
uploads[2835]

(4, datetime.datetime(2024, 7, 28, 3, 0), 181, 0.5111111111111111)

In [33]:
cursor.execute("""DROP TABLE predictions""")               

OperationalError: (1051, "Unknown table 'DBdev.predictions'")

In [35]:
# Close the cursor and connection
if cursor:
    cursor.close()
if connection:
    connection.close()
    print("Database connection closed.")

Database connection closed.
